In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", temperature=0)
reponse = llm.invoke("Hola como estas?")
reponse.content

'¡Hola! Estoy aquí y listo para ayudarte. ¿En qué puedo asistirte hoy?'

In [13]:
response = llm.invoke("Dime los productos que ofreces en la tienda")
response.content

'No tengo una tienda física ni ofrezco productos, pero puedo ayudarte a crear una lista de productos que podrías ofrecer en una tienda, o darte ideas sobre qué tipo de productos podrían ser populares. ¿Te gustaría que te ayude con eso?'

In [4]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que buscan.

Y tus productos son:
- Computadoras
- Tablets
- Celulares
- Accesorios
"""

messages = [
    ("system", system_prompt),
    ("user", "Hola, que productos ofrece la tienda?")
]

response = llm.invoke(messages)
response.content

'¡Hola! En nuestra tienda ofrecemos una variedad de productos, que incluyen:\n\n- Computadoras\n- Tablets\n- Celulares\n- Accesorios\n\nSi estás interesado en alguno de estos productos o necesitas ayuda para encontrar algo específico, ¡estaré encantado de ayudarte!'

In [11]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Busca los productos que coinciden con la consulta")
def get_products(price: int):
    """
    Busca los productos que coinciden con la consulta
    """
    products = [
        {"name": "Computadora", "price": 1000},
        {"name": "Tablet", "price": 500},
        {"name": "Celular", "price": 200},
        {"name": "Accesorio", "price": 100}
    ]
    return "".join([f"{product['name']} - {product['price']}" for product in products if product['price'] == price])


In [12]:
get_products.invoke({"price": 100})

'Accesorio - 100'

In [13]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Busca los productos que coinciden con la consulta")
def get_products():
    """
    Busca los productos que coinciden con la consulta
    """
    products = [
        {"name": "Computadora", "price": 1000},
        {"name": "Tablet", "price": 500},
        {"name": "Celular", "price": 200},
        {"name": "Accesorio", "price": 100}
    ]
    return "".join([f"{product['name']} - {product['price']}" for product in products])

In [14]:
get_products.invoke({})

'Computadora - 1000Tablet - 500Celular - 200Accesorio - 100'

In [7]:
from langchain_core.tools import tool
import requests

@tool("get_products", description="Busca los productos que coinciden con la consulta")
def get_products():
    # Connect with API
    """
    Busca los productos que coinciden con la consulta
    """
    url = "https://api.escuelajs.co/api/v1/products"
    response = requests.get(url)
    products = response.json()
    return "".join([f"{product['title']} - {product['price']}" for product in products])

In [10]:
get_products.invoke({})

'Rainbow Glitter High Heels - 41Vibrant Runners: Bold Orange & Blue Sneakers - 27Chic Summer Denim Espadrille Sandals - 33Futuristic Silver and Gold High-Top Sneaker - 68Vibrant Pink Classic Sneakers - 84Futuristic Chic High-Heel Boots - 36Elegant Patent Leather Peep-Toe Pumps with Gold-Tone Heel - 53Elegant Purple Leather Loafers - 17Sleek Futuristic Electric Bicycle - 22Classic Blue Suede Casual Shoes - 39Sleek All-Terrain Go-Kart - 37Radiant Citrus Eau de Parfum - 73Chic Transparent Fashion Handbag - 61Trendy Pink-Tinted Sunglasses - 38Elegant Glass Tumbler Set - 50cxz - 22 - 3Muka - 11New Product1764607123903updatat - 24Sleek Mirror Finish Phone Case 45 - 6767New Product1764607292576updatat - 24муки4ка - 69bag - 100iphone 17 - 1000iPhone 19 - 2000samsung 25 - 100iphone 1000 - 3000QATestLab product - 10iphone 131 - 500iPhone 199 - 2000саа - 11111New Pgd - 10New Product 1764615358763 - 12desi_test - 7Sandália - 350Tênis da Nike - 300string - 5Mishka - 5sdfd - 18hjgjkgkh - 199Mishdssf

In [17]:
from typing import Optional, List, Dict, Any
from pydantic import BaseModel, Field, confloat, conint

class SearchProductsArgs(BaseModel):

    """Parámetros de búsqueda (se filtran client-side)."""

    q: Optional[str] = Field(
        default=None,
        description="Texto a buscar dentro del título o descripción (case-insensitive)."
    )

    category_name: Optional[str] = Field(
        default=None,
        description="Nombre de categoría exacto (ej. 'Clothes')."
    )

    min_price: Optional[confloat(ge=0)] = Field(
        default=None,
        description="Precio mínimo."
    )

    max_price: Optional[confloat(ge=0)] = Field(
        default=None,
        description="Precio máximo."
    )

    limit: Optional[conint(ge=1, le=100)] = Field(
        default=10,
        description="Cantidad máxima de items a devolver (1..100)."
    )

# =========================
# Tool
# =========================

@tool(
    "busca_productos_platzi",
    args_schema=SearchProductsArgs,
    description=(
        "Busca productos en la API pública de Platzi Store y devuelve una lista de "
        "objetos con 'title', 'category' y 'price'. Aplica filtros client-side."  #Esta descripcion hace la diferencia para que el agente use la tool correctamente
    ),
)

def busca_productos_platzi(
    q: Optional[str] = None,
    category_name: Optional[str] = None,
    min_price: Optional[float] = None,
    max_price: Optional[float] = None,
    limit: int = 10,

) -> List[Dict[str, Any]]:

    """
    Devuelve una lista de dicts: [{"title": str, "category": str, "price": float}, ...]
    """

    url = "https://api.escuelajs.co/api/v1/products"

    try:
        resp = requests.get(url, timeout=15)
        resp.raise_for_status()
        products = resp.json()

    except requests.RequestException as e:
        return [{"title": "ERROR", "category": "N/A", "price": -1, "detail": str(e)}]

# Para probarlo out3 = 

busca_productos_platzi.invoke({
    "q": "tee",
    "min_price": 50,
    "limit": 5
})

In [4]:
from langchain_core.tools import tool
import requests

@tool("get_weather", description="Get the weather of the city")
def get_weather(city: str):
    response = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1")
    data = response.json()
    latitude = data["results"][0]["latitude"]
    longitude = data["results"][0]["longitude"]
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true")
    data = response.json()
    response = f"The weather in {city} is {data['current_weather']['temperature']}C with {data['current_weather']['windspeed']}km/h wind"
    return response

In [5]:
get_weather.invoke("Bogota")

'The weather in Bogota is 14.6C with 10.8km/h wind'

In [16]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que buscan y el clima de la ciudad.

Tus tools son:
- get_weather: Obtiene el clima de la ciudad
- get_products: Obtiene los productos de la tienda
"""
messages = [
    ("system", system_prompt),
    ("user", "Dime los productos que ofrece la tienda")
]
# Agregamos el bind de la coleccion de tools
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls

[{'name': 'get_products',
  'args': {},
  'id': 'call_F9RJdBCym1sjRbunQ6jeKFjO',
  'type': 'tool_call'}]

In [17]:
system_prompt = """
Eres un asistente de ventas que ayuda a los clientes a encontrar los productos que buscan y el clima de la ciudad.

Tus tools son:
- get_weather: Obtiene el clima de la ciudad
- get_products: Obtiene los productos de la tienda
"""
messages = [
    ("system", system_prompt),
    ("user", "Cual es el clima de la capital de Colombia?")
]
# Agregamos el bind de la coleccion de tools
llm_with_tools = llm.bind_tools([get_products, get_weather])
response = llm_with_tools.invoke(messages)
response.tool_calls

[{'name': 'get_weather',
  'args': {'city': 'Bogotá'},
  'id': 'call_yYTbbNLnTGqzINAc9db1ySUg',
  'type': 'tool_call'}]